## THE BATTLE OF NEIGHBORHOODS FROM COURSERA CAPSTONE(Johannesburg)

### Introduction
#### The problem:
This project is related to understanding the appropriate location for having a gas station in **South Africa**. The city of choice is  **Johannesburg**. **Johannesburg** is the largest city of **South Africa** and is also one of the 50 largest urban areas of the world. Apart from being the largest city, **Johannesburg** is also most populous city with a population density of >2000 /sq km. The urban population is also very high. The passenger car sales for the past two months(Dec-19 and Jan-20)  was averaging at ~40000 cars in the country with a major contribution from **Johannesburg**.
The purpose of this work is to locate an appropriate place for Gas station in **Johannesburg**. The existing Gas Stations locations and their proximity to each other will be analyzed as part of this work.
### Data
Based on definition of our problem, factors that will influence our decission are:

- 	List of location and neighborhoods of Johannesburg 
- 	Number of and distance other Gas stations in  the neighborhood, if any

### Data Sources
- 	Municipalities  and neighborhoods of Johannesburg from Wikipedia (https://en.wikipedia.org/wiki/City_of_Johannesburg_Metropolitan_Municipality);
- 	Geocode information from Geopy;
- 	Gas stations in Johannesburg from Foursquare


Based on definition of our problem, factors that will influence our decision are:
- 	number of existing restaurants in the neighborhood (any type of restaurant)
- 	number of and distance to Italian restaurants in the neighborhood, if any
- 	distance of neighborhood from city center



In [1]:
#Import the required libraries
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#Define the url and get the table data from the website url
#The data will be stored in My_table
website_url = requests.get('https://en.wikipedia.org/wiki/City_of_Johannesburg_Metropolitan_Municipality').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [3]:
#Define the dataframe and extract the data from html table into the dataframe
column_names = ['Place','Code', 'Population','Area'] 
SAdf=pd.DataFrame(columns=column_names)

rows=My_table.find_all('tr')
res=[]
for row in rows:
    cells= row.find_all('td')
    res.append(cells)

for i in range(len(res)):
    if (i!=0) :
        SAdf = SAdf.append({'Place': res[i][0].text,'Code': res[i][1].text,'Population': res[i][2].text,'Area': res[i][3].text}, ignore_index=True)

In [4]:
SAdf_withlat=SAdf.copy()

In [6]:
address = 'Johannesburg'

geolocator = Nominatim(user_agent="SA_Exploror")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Johannesburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Johannesburg are -26.205, 28.049722.


In [7]:
def getaddress(address):
    try:
        
        geolocator = Nominatim(user_agent="SA_Exploror")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        return(latitude,longitude )
    except:
        return(None,None)


In [9]:
SAdf_withlat['latitude']=0
SAdf_withlat['longitude']=0

for index,rec in SAdf_withlat.iterrows():
    address =rec["Place"] +", South Africa"
    SAdf_withlat.iloc[index,4],SAdf_withlat.iloc[index,5]=getaddress(address)
    

In [78]:


SAdf_withlat["Area"] = SAdf_withlat.Area.astype(float)
SAdf_withlat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
Place         35 non-null object
Code          35 non-null object
Population    35 non-null object
Area          35 non-null float64
latitude      35 non-null float64
longitude     35 non-null float64
dtypes: float64(3), object(3)
memory usage: 1.7+ KB


In [215]:
#SAdf_withlat.drop(SAdf_withlat[SAdf_withlat.latitude !='NaN'].index, inplace=True)
SAdf_withlat.drop(SAdf_withlat[(SAdf_withlat['latitude'].isna())].index,inplace = True)
SAdf_withlat.reset_index(inplace=True, drop=True)




In [80]:
SAdf_withlat.sort_values(by='Area',ascending=False).head()

,Place,Code,Population,Area,latitude,longitude
8,Johannesburg,798015,"957,441",334.81,-26.205000,28.049722
29,Soweto,798026,"1,271,628",200.03,-26.222778,27.890000
24,Randburg,798016,"337,053",167.98,-26.096111,27.975000
27,Roodepoort,798022,"326,416",161.50,-26.156389,27.885833
20,Midrand,798004,"87,387",152.87,-25.999262,28.125912


In [14]:
print('The geograpical coordinate of Johannesberg are {}, {}.'.format(latitude, longitude))
    

The geograpical coordinate of Johannesberg are -26.205, 28.049722.


In [15]:
address = 'Johannesberg'

geolocator = Nominatim(user_agent="SA_Exploror")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Johannesberg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Johannesberg are -26.205, 28.049722.


In [31]:
map_SA= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(SAdf_withlat['latitude'], SAdf_withlat['longitude'], SAdf_withlat['Place']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SA)  
map_SA



    

#### Define Foursquare Credentials and Version
    

In [33]:
SAdf_withlat.loc[8, 'Place']

'Johannesburg'

In [34]:
neighborhood_latitude = SAdf_withlat.loc[8, 'latitude'] # neighborhood latitude value
neighborhood_longitude = SAdf_withlat.loc[8, 'longitude'] # neighborhood longitude value

neighborhood_name = SAdf_withlat.loc[8, 'Place'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Johannesburg are -26.205, 28.049722.


#### Use the categoryid of Gas stations to list only the gas stations

In [54]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#Category id of Gas stations  is 
categoryId = '4bf58dd8d48988d113951735'


radius = 10000 # define radius

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    categoryId)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=JSIFITKBBUMWHQIVML0FPPQDET4FD1TQ4PIAVVJI4G3NKRVM&client_secret=GKTGD43YEMMTUNIENCLNIWN5SIMZGTFBN5EYV440M1Z3B3HS&v=20180605&ll=-26.205,28.049722&radius=10000&limit=100&categoryId=4bf58dd8d48988d113951735'

'https://api.foursquare.com/v2/venues/search?&client_id=JSIFITKBBUMWHQIVML0FPPQDET4FD1TQ4PIAVVJI4G3NKRVM&client_secret=GKTGD43YEMMTUNIENCLNIWN5SIMZGTFBN5EYV440M1Z3B3HS&v=20180605&ll=-26.205,28.049722&radius=10000&limit=100&categoryId=4bf58dd8d48988d113951735'

In [56]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e59771e1e152c001bd0a2e7'},
 'response': {'venues': [{'id': '55cd09ab498ebb16372df295',
    'name': 'BP',
    'location': {'address': 'Bram Fischer Dr & 1st Ave',
     'lat': -26.12608016,
     'lng': 27.9961842,
     'labeledLatLngs': [{'label': 'display',
       'lat': -26.12608016,
       'lng': 27.9961842}],
     'distance': 10285,
     'postalCode': '2123',
     'cc': 'ZA',
     'city': 'Randburg',
     'state': 'IGauteng',
     'country': 'iNingizimu Afrika',
     'formattedAddress': ['Bram Fischer Dr & 1st Ave',
      'Randburg',
      '2123',
      'iNingizimu Afrika']},
    'categories': [{'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'shortName': 'Gas Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1582921551',
    'hasPerk': False},
   {'id': '4d67f5f0709bb60c4c40b014',


In [58]:
nearby_venues=json_normalize(results['response']['venues'])
    

In [59]:

filtered_columns = ['name',  'location.lat', 'location.lng','location.distance','location.city','location.state']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [62]:
nearby_venues.shape

(50, 6)

In [64]:
map_Johan= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['location.lat'], nearby_venues['location.lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Johan)  
map_Johan


In [65]:
gas_station_data= nearby_venues.copy()

In [66]:
gas_station_data.head()

,name,location.lat,location.lng,location.distance,location.city,location.state
0,BP,-26.126080,27.996184,10285,Randburg,IGauteng
1,KFC,-26.268549,27.977463,10104,EGoli,IGauteng
2,Engen Parktown Convenience Centre,-26.183036,28.036445,2781,EGoli,IGauteng
3,BP,-26.116123,28.106763,11417,Bramley View,IGauteng
4,Sasol Van Buuren,-26.187960,28.135535,8778,Germiston,IGauteng


In [67]:
gas_station_data.groupby('name').count()

,location.lat,location.lng,location.distance,location.city,location.state
name,,,,,
BP,13,13,13,13,13
BP Empire,1,1,1,1,1
Caltex Universal Motors,1,1,1,1,1
"Caltex, Cyrildene, Johannesburg, Gauteng, ZA/ South Africa",1,1,1,1,1
Engen,4,4,4,3,3
Engen - Ellis Park Convenience Centre,1,1,1,1,1
Engen - Northcliff Corner Convenience Centre,1,1,1,1,1
Engen 1plus,1,1,1,1,1
Engen Glenvista,1,1,1,1,1


### The Data shows that Shell , BP Engen has major presence in the city followed by Total and Sasol and others

In [68]:
gas_station_data.groupby('location.city').count()

,name,location.lat,location.lng,location.distance,location.state
location.city,,,,,
Alberton,1,1,1,1,1
Bedfordview,2,2,2,2,2
Bramley View,1,1,1,1,1
EGoli,22,22,22,22,22
Edenvale,1,1,1,1,1
Emmarentia,1,1,1,1,1
Emmerentia,1,1,1,1,1
"Fairlands, Johannesburg",1,1,1,1,1
Germiston,4,4,4,4,4


In [70]:
gas_station_data['location.distance'].describe()

count       50.000000
mean      7510.560000
std       2837.779537
min       1908.000000
25%       5776.000000
50%       7507.500000
75%       9377.750000
max      13890.000000
Name: location.distance, dtype: float64

In [71]:
gas_station_data['location.city'].value_counts()

EGoli                      22
Germiston                   4
Randburg                    2
Roodepoort                  2
Bedfordview                 2
Weltevreden Park            1
Emmerentia                  1
Victory Park                1
Melville                    1
Fairlands, Johannesburg     1
Sandhurst                   1
Kensington                  1
Emmarentia                  1
Killarney                   1
Bramley View                1
Edenvale                    1
Alberton                    1
Johannesburg South          1
Rosebank                    1
Name: location.city, dtype: int64